<a href="https://colab.research.google.com/github/EsSanches/DE-step/blob/main/Task_spark_2_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql.functions import col, concat, lit, when, rand, current_date, date_add, date_format
import os
from inspect import currentframe
from datetime import datetime, timedelta


spark = SparkSession.builder.appName("task_spark_2").getOrCreate()


# Ввод числа генерируемых данных
while True:
    try:
        u_input = int(input("Введите положительное, целое число: "))
        if u_input < 1:
            raise Exception()
        print("Вы ввели", u_input)
        break
    except Exception as e:
        print('Неверный формат')


# заполнение и создание колонок id, name, email, city, age, salary, registration_date)
date = [(i, f"Name_{i}", f"City_n_{i}") for i in range(1, u_input + 1)]
df = spark.createDataFrame(date, ["id", "name", "city"])
df.show()


#добавление колонок
df = df.withColumn("email", concat(col("name"), lit("@example."), when(col("id") % 2 == 0, "ru").otherwise("com")))\
  .withColumn("age", (rand() * 75 + 18).cast("int")).withColumn("salary", (rand() * 100000 + 20000).cast("int"))\
  .withColumn("registration_date", date_add(current_date(), - (rand() * (col("age") - 18)).cast("int") * 365))


# Замена 5% данных на значение NULL
columns = ["name", "email", "city", "age", "salary", "registration_date"]
df = df.select("id", *[when(rand() <= 0.05, None).otherwise(col(column)).alias(column) for column in columns])
df.show()


# Установка текущей даты для имени файла
df_date = spark.sql("SELECT current_date()")
current_date = df_date.select(date_format("current_date", "yyyy-MM-dd")).first()[0]


# получаем текущую дату и время
cur_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# сохряняем датафрейм в единый файл
output_path = f"{cur_date}-dev.csv"
df.coalesce(1).write.mode("overwrite").csv(output_path, header=True)

# переименовываем csv файл
csv_file = [f for f in os.listdir(output_path) if f.endswith('.csv')]
old_file_name = csv_file[0]
old_file_path = os.path.join(output_path, old_file_name)
new_file_name = f"{cur_date}-dev.csv"
new_file_path = os.path.join(output_path, new_file_name)

os.rename(old_file_path, new_file_path)

print(f'CSV файл сохранен по пути: {output_path}')